In [7]:
import SimpleITK as sitk
import nibabel as nib
import numpy as np
import os

In [2]:
# 最大连通域提取
def GetLargestConnectedCompont(binarysitk_image):
    cc = sitk.ConnectedComponent(binarysitk_image)
    stats = sitk.LabelIntensityStatisticsImageFilter()
    stats.SetGlobalDefaultNumberOfThreads(8)
    stats.Execute(cc, binarysitk_image)
    maxlabel = 0
    maxsize = 0
    for l in stats.GetLabels():
        size = stats.GetPhysicalSize(l)
        if maxsize < size:
            maxlabel = l
            maxsize = size
    labelmaskimage = sitk.GetArrayFromImage(cc)
    outmask = labelmaskimage.copy()
    outmask[labelmaskimage == maxlabel] = 255
    outmask[labelmaskimage != maxlabel] = 0
    outmask_sitk = sitk.GetImageFromArray(outmask)
    outmask_sitk.SetDirection(binarysitk_image.GetDirection())
    outmask_sitk.SetSpacing(binarysitk_image.GetSpacing())
    outmask_sitk.SetOrigin(binarysitk_image.GetOrigin())
    return outmask_sitk

# 逻辑与操作
def GetMaskImage(sitk_src, sitk_mask, replacevalue=0):
    array_src = sitk.GetArrayFromImage(sitk_src)
    array_mask = sitk.GetArrayFromImage(sitk_mask)
    array_out = array_src.copy()
    array_out[array_mask == 0] = replacevalue
    outmask_sitk = sitk.GetImageFromArray(array_out)
    outmask_sitk.SetDirection(sitk_src.GetDirection())
    outmask_sitk.SetSpacing(sitk_src.GetSpacing())
    outmask_sitk.SetOrigin(sitk_src.GetOrigin())
    return outmask_sitk

In [5]:
image = nib.load(".\data\RibFrac501-image.nii.gz")
im_affine = image.affine
image = image.get_fdata().astype(np.int16)
image = sitk.GetImageFromArray(image.astype('int16'))

In [10]:
tmp_path = ".\\data\\tmp"

In [28]:
sitk_seg = sitk.BinaryThreshold(image, lowerThreshold=250, upperThreshold=3000, insideValue=255, outsideValue=0)
#first fit 阈值200
#分出胸椎，肋骨很好 阈值300
sitk.WriteImage(sitk_seg, os.path.join(tmp_path,'step1.mha'))

In [29]:
sitk_open = sitk.BinaryMorphologicalOpening(sitk_seg != 0, (2,2,2))
sitk_open = GetLargestConnectedCompont(sitk_open)
sitk.WriteImage(sitk_open, os.path.join(tmp_path,'step2.mha'))

In [30]:
array_open = sitk.GetArrayFromImage(sitk_open)
array_seg = sitk.GetArrayFromImage(sitk_seg)
array_mask = array_seg - array_open
sitk_mask = sitk.GetImageFromArray(array_mask)
sitk_mask.SetDirection(sitk_seg.GetDirection())
sitk_mask.SetSpacing(sitk_seg.GetSpacing())
sitk_mask.SetOrigin(sitk_seg.GetOrigin())
sitk.WriteImage(sitk_mask, os.path.join(tmp_path,'step3.mha'))

In [31]:
skeleton_mask = GetLargestConnectedCompont(sitk_mask)
sitk.WriteImage(skeleton_mask, os.path.join(tmp_path,'step4.mha'))

In [32]:
sitk_skeleton = GetMaskImage(image, skeleton_mask, replacevalue=-1500)
sitk.WriteImage(sitk_skeleton,  os.path.join(tmp_path,'step5.mha'))